# MDL Assignment 3 Part 2:

In [1]:
rollno1 = 2019101030
rollno2 = 2019101081

x = 1 - ((rollno1%10**4)%30 + 1)/100
success_reward = rollno1%90 + 10

In [2]:
def mapper(position):
    row , col = position
    x = col
    y = row
    return(x, y)

In [3]:
X = [0, 1, 2, 3]
Y = [0, 1]
call = [0, 1]

X_MAX = 3
Y_MAX = 1

ALL_STATES = []

for x1 in X:
    for y1 in Y:
        for x2 in X:
            for y2 in Y:
                for state in call:
                    ALL_STATES.append(((x1,y1),(x2,y2),state))

ACTION_STAY = 'Stay'
ACTION_UP = 'Up'
ACTION_DOWN = 'Down'
ACTION_LEFT = 'Left'
ACTION_RIGHT = 'Right'

ACTIONS = [ACTION_STAY, ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]

O1 = "o1"
O2 = "o2"
O3 = "o3"
O4 = "o4"
O5 = "o5"
O6 = "o6"

OBSERVATIONS = [O1, O2, O3, O4, O5, O6]

In [4]:
## Transition Matrix

def transition(state, action):
    global success_reward
    a_pos , t_pos, call = state

    new_states = []
    probabilities = [] 
    rewards = []

    if(action == ACTION_LEFT):
        # success
        if(a_pos[0] == 0):
            new_a_pos = (a_pos[0], a_pos[1])
        else:
            new_a_pos = (a_pos[0] - 1, a_pos[1])

        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(x)
        rewards.append(-1)

        # failure
        if(a_pos[0] == X_MAX):
            new_a_pos = (a_pos[0], a_pos[1])
        else:
            new_a_pos = (a_pos[0] + 1, a_pos[1])

        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(1-x)
        rewards.append(-1)

    elif(action == ACTION_RIGHT):
        # success
        if(a_pos[0] == X_MAX):
            new_a_pos = (a_pos[0], a_pos[1])
        else:
            new_a_pos = (a_pos[0] + 1, a_pos[1])

        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(x)
        rewards.append(-1)

        # failure
        if(a_pos[0] == 0):
            new_a_pos = (a_pos[0], a_pos[1])
        else:
            new_a_pos = (a_pos[0] - 1, a_pos[1])

        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(1-x)
        rewards.append(-1)


    elif(action == ACTION_UP):
        # success
        if(a_pos[1] == Y_MAX):
            new_a_pos = (a_pos[0], a_pos[1])
        else:
            new_a_pos = (a_pos[0], a_pos[1]+1)

        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(x)
        rewards.append(-1)

        # failure
        if(a_pos[1] == 0):
            new_a_pos = (a_pos[0], a_pos[1])
        else:
            new_a_pos = (a_pos[0], a_pos[1]-1)

        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(1-x)
        rewards.append(-1)


    elif(action == ACTION_DOWN):
        # success
        if(a_pos[1] == 0):
            new_a_pos = (a_pos[0], a_pos[1])
        else:
            new_a_pos = (a_pos[0], a_pos[1]-1)

        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(x)
        rewards.append(-1)

        # failure
        if(a_pos[1] == Y_MAX):
            new_a_pos = (a_pos[0], a_pos[1])
        else:
            new_a_pos = (a_pos[0], a_pos[1]+1)

        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(1 - x)
        rewards.append(-1)
        
    else:
        #STAY
        new_a_pos = (a_pos[0], a_pos[1])
        new_states.append((new_a_pos, t_pos, call))
        probabilities.append(1)
        rewards.append(0)
 



    l = len(new_states)
    for i in range(l):
        prob = probabilities[i]
        a_pos, t_pos, call = new_states[i]
        reward = rewards[i]

        ## stays (dont append):
        probabilities[i] = prob * 0.6


        ## right
        if(t_pos[0] == X_MAX):
            new_t_pos = (t_pos[0], t_pos[1])
        else:
            new_t_pos = (t_pos[0] + 1, t_pos[1])

        new_states.append((a_pos, new_t_pos, call))
        probabilities.append(0.1 * prob)
        rewards.append(reward)

        ## left
        if(t_pos[0] == 0):
            new_t_pos = (t_pos[0], t_pos[1])
        else:
            new_t_pos = (t_pos[0] - 1, t_pos[1])

        new_states.append((a_pos, new_t_pos, call))
        probabilities.append(0.1 * prob)
        rewards.append(reward)

        ## up
        if(t_pos[1] == Y_MAX):
            new_t_pos = (t_pos[0], t_pos[1])
        else:
            new_t_pos = (t_pos[0], t_pos[1]+1)

        new_states.append((a_pos, new_t_pos, call))
        probabilities.append(0.1 * prob)
        rewards.append(reward)

        ## down 
        if(t_pos[1] == 0):
            new_t_pos = (t_pos[0],t_pos[1])
        else:
            new_t_pos = (t_pos[0], t_pos[1]-1)

        new_states.append((a_pos, new_t_pos, call))
        probabilities.append(0.1 * prob)
        rewards.append(reward)
       

    l = len(new_states)

    for i in range(l):
        a_pos, t_pos, call = new_states[i]
        prob = probabilities[i]
        
        # Agent reached Target => Call turned off
        if(a_pos == t_pos) and call == 1:
            new_states[i] = (a_pos, t_pos, 0)
            rewards[i] += success_reward
        
        # Call is on
        elif call == 1:
            # Call remains on
            probabilities[i] = prob*0.9

            # Call turns off
            new_states.append((a_pos, t_pos, 0))
            probabilities.append(prob*0.1)
            rewards.append(rewards[i])

        # Call is off
        else:
            # Call remains off
            probabilities[i] = prob*0.5

            # Call turn on
            new_states.append((a_pos, t_pos, 1))
            probabilities.append(prob*0.5)
            rewards.append(rewards[i])


    
    final_states = []
    final_probabilities = []
    final_rewards = []

    l = len(new_states)
    for i in range(l):
        if(new_states[i] in final_states):
            final_probabilities[final_states.index(new_states[i])] += probabilities[i]
        else:
            final_states.append(new_states[i])
            final_probabilities.append(probabilities[i])
            final_rewards.append(rewards[i])

    return final_states, final_probabilities, final_rewards

In [5]:
def observation(end_state):

    pos1, pos2, status = end_state
    x1,y1 = pos1
    x2,y2 = pos2

    if x1 == x2 and y1 == y2:
        return O1
    
    elif x1+1 == x2 and y1 == y2:
        return O2

    elif x1 == x2 and y1-1 == y2:
        return O3

    elif x1-1 == x2 and y1 == y2:
        return O4
    
    elif x1 == x2 and y1+1 == y2:
        return O5

    else:
        return O6

In [6]:
def parse(state):
    a_pos, t_pos, call  = state
    return "S-" + str(a_pos[0]) + str(a_pos[1])+ "-" + str(t_pos[0]) + str(t_pos[1]) + "-" + str(call) 

In [7]:
belief_state_q1 = []
check_var = mapper((1,0))

for i in ALL_STATES:
    pos1, pos2, state = i
    if pos2 == check_var and (observation(i) == O6):
        belief_state_q1.append(i)

In [8]:
belief_state_q2 = []
check_var = mapper((1,1))
for i in ALL_STATES:
    pos1, pos2, state = i
    if pos1 == (1, 1) and (observation(i) == O1 or observation(i) == O2  or observation(i) == O3 or observation(i) == O4 or observation(i) == O5) and i[2] == 0:
        belief_state_q2.append(i)

In [9]:
belief_state_q4 = []
for i in ALL_STATES:
    pos1, pos2, state = i


    ### all the coordinates are mapped manually

    if(pos1 == (0,0)):
        if pos2 == (1,0):
            belief_state_q4.append(0.1/2)
        elif pos2 == (2,0):
            belief_state_q4.append(0.1/2)
        elif pos2 == (1,1):
            belief_state_q4.append(0.1/2)
        elif pos2 == (2,1):
            belief_state_q4.append(0.1/2)
        else:
            belief_state_q4.append(0)
    
    elif (pos1 == (3,1)):
        if pos2 == (1,0):
            belief_state_q4.append(0.15/2)
        elif pos2 == (2,0):
            belief_state_q4.append(0.15/2)
        elif pos2 == (1,1):
            belief_state_q4.append(0.15/2)
        elif pos2 == (2,1):
            belief_state_q4.append(0.15/2)
        else:
            belief_state_q4.append(0)

    else:
        belief_state_q4.append(0)


In [10]:
f = open('file.pomdp','w')
f.write("discount: 0.5\n")
f.write("values: reward\n")

f.write("states: ")
for i in ALL_STATES:
    f.write(str(parse(i)) + " ")
f.write("\n")

f.write("actions: ")
for i in ACTIONS:
    f.write(str(i) + " ")
f.write("\n")

f.write("observations: ")
for i in OBSERVATIONS:
    f.write(str(i) + " ")
f.write("\n")

1

In [11]:
f.write("start include: ")
for i in belief_state_q1:
    f.write(str(parse(i)) + " ")
f.write("\n")

1

In [12]:
## "T: <action> : <start-state> : <end-state> %f"
for action in ACTIONS:
    for state in ALL_STATES:
        new_states, probabilities, rewards = transition(state, action)

        l = len(new_states)
        for i in range(l):
            f.write("T : " + action + " : " + str(parse(state)) + " : " +str(parse(new_states[i])) + " "  + "%1.5f"%(probabilities[i]) + "\n") 


In [13]:
for i in ALL_STATES:
    f.write("O : * : "+str(parse(i))+" : "+observation(i)+" 1.0 \n")

In [14]:

for action in ACTIONS:
    for state in ALL_STATES:
        new_states, probabilities, rewards = transition(state, action)

        l = len(new_states)
        for i in range(l):
            f.write("R : " + action + " : " + str(parse(state)) + " : " +str(parse(new_states[i])) + " : * "  + "%f"%(rewards[i]) + "\n") 

f.close()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=10c6ce1f-e206-4ca6-a148-0fe16f2faac5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>